# 1. Preamble

In [1]:
import pandas
import math
import re
import networkx as nx
import networkx.algorithms
import sklearn
import sklearn.cluster
import sklearn.preprocessing
import matplotlib.pyplot as plt

# 2. Data Import and Exploration

## 2.1 First Dataset ( Company and Directors )

In [2]:
company_director_raw = pandas.read_csv('/content/company_directorships.csv')
company_director_raw.head(5)

,company_name,cikcode,director_name,software_background,start_date,end_date
0,1ST SOURCE CORP,34782,ALLISON N. EGIDI,f,2011-03-14,2017-03-14
1,1ST SOURCE CORP,34782,ANDREA G. SHORT,f,2023-03-10,2025-03-14
2,1ST SOURCE CORP,34782,CHRISTOPHER J. MURPHY III,t,2008-03-14,2025-03-14
3,1ST SOURCE CORP,34782,CHRISTOPHER J. MURPHY IV,f,2011-03-14,2025-03-14
4,1ST SOURCE CORP,34782,CRAIG A. KAPSON,f,2008-03-14,2017-03-14


In [3]:
company_director_raw.info()
missing_values = company_director_raw.isnull().sum()
print("Missing values in each column:\n", missing_values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13347 entries, 0 to 13346
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   company_name         13347 non-null  object
 1   cikcode              13347 non-null  int64 
 2   director_name        13347 non-null  object
 3   software_background  13347 non-null  object
 4   start_date           13347 non-null  object
 5   end_date             13347 non-null  object
dtypes: int64(1), object(5)
memory usage: 625.8+ KB
Missing values in each column:
 company_name           0
cikcode                0
director_name          0
software_background    0
start_date             0
end_date               0
dtype: int64


## 2.2 Second Dataset ( Director Details )

In [4]:
director_details_raw = pandas.read_csv('/content/director-details.csv')
director_details_raw.sample(5)

,id,url,name,age,role,compensation,source_excerpt,created_at,gender
1891,1879,https://www.sec.gov/Archives/edgar/data/47111/...,Timothy W. Curoe,55,"Chief Executive Officer, R.D. Offutt Company",0,Timothy W. Curoe\nDirector Nominee\nTerm 0 yea...,2025-05-16 16:31:45.845946,male
3714,3707,https://www.sec.gov/Archives/edgar/data/99302/...,Dawn DePerrior,66,Independent Director,444300,Ms. DePerrior retired as a managing director i...,2025-05-16 16:34:20.596167,female
3701,3694,https://www.sec.gov/Archives/edgar/data/98362/...,"Ward J. Timken, Jr.",57,"Chief Executive Officer, McKinley Strategies, LLC",252632,"Ward J. Timken, Jr., Age: 57, Director since 2...",2025-05-16 16:34:18.04527,male
2686,2670,https://www.sec.gov/Archives/edgar/data/70487/...,Donald M. Berwick,77,Director,150006,"Donald M. Berwick, 77, has served as a directo...",2025-05-16 16:32:52.924623,male
3734,3727,https://www.sec.gov/Archives/edgar/data/99780/...,Leldon E. Echols,69,Non-Executive Chairman of the Board,493088,MR. ECHOLS serves as non-executive Chairman of...,2025-05-16 16:34:22.221808,male


In [5]:
director_details_raw.info()
missing_values = director_details_raw.isnull().sum()
print("Missing values in each column:\n", missing_values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5910 entries, 0 to 5909
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5910 non-null   int64 
 1   url             5910 non-null   object
 2   name            5910 non-null   object
 3   age             5910 non-null   int64 
 4   role            5909 non-null   object
 5   compensation    5910 non-null   int64 
 6   source_excerpt  5910 non-null   object
 7   created_at      5910 non-null   object
 8   gender          5889 non-null   object
dtypes: int64(3), object(6)
memory usage: 415.7+ KB
Missing values in each column:
 id                 0
url                0
name               0
age                0
role               1
compensation       0
source_excerpt     0
created_at         0
gender            21
dtype: int64


## 2.3 Third Dataset (Task 4)

In [6]:
top_companies_raw = pandas.read_csv('/content/fortune1000_2024.csv')
top_companies_raw.sample(5)

,Rank,Company,Ticker,Sector,Industry,Profitable,Founder_is_CEO,FemaleCEO,Growth_in_Jobs,Change_in_Rank,...,Assets_M,CEO,Country,HeadquartersCity,HeadquartersState,Website,CompanyType,Footnote,MarketCap_Updated_M,Updated
72,73,Allstate,ALL,Financials,Insurance: Property and Casualty (Stock),no,no,no,no,11.0,...,103362.0,Thomas J. Wilson,U.S.,Northbrook,Illinois,https://www.allstate.com,Public,"Market value as of July 15, 2024.",44050.0,2024-08-05
7,8,Alphabet,GOOGL,Technology,Internet Services and Retailing,yes,no,no,no,0.0,...,402392.0,Sundar Pichai,U.S.,Mountain View,California,https://www.abc.xyz,Public,"Market value as of July 15, 2024.",2315182.0,2024-08-05
917,918,PC Connection,CNXN,Wholesalers,Wholesalers: Electronics and Office Equipment,yes,no,no,yes,-42.0,...,1188.4,Timothy J. McGrath,U.S.,Merrimack,New Hampshire,https://www.connection.com,Public,"Market value as of March 28, 2024.",1738.0,2024-06-04
497,498,Super Micro Computer,SMCI,Technology,"Computers, Office Equipment",yes,yes,no,yes,138.0,...,3674.7,Charles Liang,U.S.,San Jose,California,https://www.supermicro.com,Public,"Figures are for fiscal year ended June 30, 202...",59138.0,2024-06-04
473,474,Williams-Sonoma,WSM,Retailing,Specialty Retailers: Other,yes,no,yes,no,-37.0,...,5273.5,Laura J. Alber,U.S.,San Francisco,California,https://www.williams-sonomainc.com,Public,"Figures are for fiscal year ended Jan. 31, 202...",20358.0,2024-06-04


In [7]:
top_companies_raw.info()
missing_values = top_companies_raw.isnull().sum()
print("Missing values in each column:\n", missing_values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Rank                           1000 non-null   int64  
 1   Company                        1000 non-null   object 
 2   Ticker                         959 non-null    object 
 3   Sector                         1000 non-null   object 
 4   Industry                       1000 non-null   object 
 5   Profitable                     1000 non-null   object 
 6   Founder_is_CEO                 1000 non-null   object 
 7   FemaleCEO                      1000 non-null   object 
 8   Growth_in_Jobs                 1000 non-null   object 
 9   Change_in_Rank                 1000 non-null   float64
 10  Gained_in_Rank                 1000 non-null   object 
 11  Dropped_in_Rank                1000 non-null   object 
 12  Newcomer_to_the_Fortune500     500 non-null    ob

# 3. Data Cleaning and Preprocessing

## 3.1 Cleaning the names

In [8]:
def clean_name(name):
    name = re.sub(r'[^A-Z ]', ' ', name.upper())  # keep only letters and spaces
    name = re.sub(r'\s+', ' ', name)              # collapse multiple spaces into one
    return name.strip()

In [9]:
company_director_raw['company_name'] = company_director_raw['company_name'].apply(clean_name)
company_director_raw['director_name'] = company_director_raw['director_name'].apply(clean_name)

director_details_raw['name'] = director_details_raw['name'].apply(clean_name)

top_companies_raw['Company'] = top_companies_raw['Company'].apply(clean_name)

In [10]:
top_companies_raw.rename(columns={'Company': 'company_name'}, inplace=True)
director_details_raw.rename(columns={'name': 'director_name'}, inplace=True)

In [11]:
company_director_raw.sample(5)

,company_name,cikcode,director_name,software_background,start_date,end_date
3919,EMERSON RADIO CORP,32621,STEPHEN H GOODMAN,f,2002-07-29,2003-07-29
3194,CURTISS WRIGHT CORP,26324,LARRY D WYCHE,f,2024-03-22,2025-03-27
7952,MAUI LAND PINEAPPLE CO INC,63330,ANTHONY P TAKITANI,f,2015-03-13,2025-04-07
525,AMERICAN AIRLINES GROUP INC,6201,MATTHEW J HART,f,2015-04-22,2024-04-25
7564,LEVI STRAUSS CO,94845,YAEL GARTEN,t,2020-02-27,2025-03-12


## 3.2 Feature Engineering

In [12]:
company_director_raw['service_years'] = round((pandas.to_datetime(company_director_raw.end_date) - pandas.to_datetime(company_director_raw.start_date)).dt.days / 365, 2)
display(company_director_raw.head())
print(company_director_raw.size)

,company_name,cikcode,director_name,software_background,start_date,end_date,service_years
0,ST SOURCE CORP,34782,ALLISON N EGIDI,f,2011-03-14,2017-03-14,6.01
1,ST SOURCE CORP,34782,ANDREA G SHORT,f,2023-03-10,2025-03-14,2.01
2,ST SOURCE CORP,34782,CHRISTOPHER J MURPHY III,t,2008-03-14,2025-03-14,17.01
3,ST SOURCE CORP,34782,CHRISTOPHER J MURPHY IV,f,2011-03-14,2025-03-14,14.01
4,ST SOURCE CORP,34782,CRAIG A KAPSON,f,2008-03-14,2017-03-14,9.01


93429


In [13]:
company_director = pandas.DataFrame({
    'service_years': company_director_raw.groupby(['director_name', 'company_name']).service_years.sum(),
}).reset_index()
display(company_director.head())
print(company_director.size)

,director_name,company_name,service_years
0,A A BUSCH III,EMERSON ELECTRIC CO,2.01
1,A ALEXANDER ARNOLD III,ACCELERATE DIAGNOSTICS INC,2.99
2,A ALEXANDER MCLEAN III,WORLD ACCEPTANCE CORP,6.02
3,A BARRY RAND,CAMPBELL S CO,1.02
4,A BART HOLADAY,MDU RESOURCES GROUP INC,7.04


39576


In [14]:
top_company_set = set(top_companies_raw['company_name'])
company_director['is_top_company'] = company_director['company_name'].isin(top_company_set)
company_director

,director_name,company_name,service_years,is_top_company
0,A A BUSCH III,EMERSON ELECTRIC CO,2.01,1
1,A ALEXANDER ARNOLD III,ACCELERATE DIAGNOSTICS INC,2.99,1
2,A ALEXANDER MCLEAN III,WORLD ACCEPTANCE CORP,6.02,1
3,A BARRY RAND,CAMPBELL S CO,1.02,1
4,A BART HOLADAY,MDU RESOURCES GROUP INC,7.04,1
...,...,...,...,...
13187,ZHONGLI LIU,SMART POWERR CORP,4.01,1
13188,ZI YAO LIM,KULICKE SOFFA INDUSTRIES INC,0.00,1
13189,ZIV SHOSHANI,VISHAY INTERTECHNOLOGY INC,15.99,1
13190,ZUHEIR SOFIA,LANCASTER COLONY CORP,17.99,1


In [15]:
company_director.groupby('is_top_company').count()

,director_name,company_name,service_years
is_top_company,,,
1,12991,12991,12991
1000,201,201,201


In [16]:
def role_importance(role):
    if pandas.isna(role): return 1
    role = role.lower()
    if 'chief executive officer' in role or 'ceo' in role:
        return 10
    elif 'chair' in role:
        return 7
    elif 'president' in role:
        return 5
    elif 'director' in role:
        return 3
    else:
        return 1

In [17]:
director_details_raw['role_score'] = director_details_raw['role'].map(role_importance)
director_details = pandas.DataFrame({
    'compensation': director_details_raw.groupby('director_name').compensation.sum(),
    'role_score': director_details_raw.groupby('director_name').role_score.max(),
})
director_details['log_compensation'] = (1 + director_details.compensation).map(math.log10)
director_details.drop('compensation', axis=1, inplace=True)
director_details

,role_score,log_compensation
director_name,,
A CATHERINE NGO,3,6.002665
A EUGENE WASHINGTON,3,5.511712
A F PETROCELLI,10,4.672107
A FARAZ ABBASI,3,5.055501
A G SULZBERGER,7,6.693304
...,...,...
ZACKARY IRANI,10,5.556428
ZAHID AFZAL,3,0.000000
ZENA SRIVATSA ARNOLD,3,5.278664


In [18]:
company_directors_details = pandas.merge(company_director, director_details, on='director_name', how='left').fillna(0)
company_directors_details

,director_name,company_name,service_years,is_top_company,role_score,log_compensation
0,A A BUSCH III,EMERSON ELECTRIC CO,2.01,1,0.0,0.000000
1,A ALEXANDER ARNOLD III,ACCELERATE DIAGNOSTICS INC,2.99,1,0.0,0.000000
2,A ALEXANDER MCLEAN III,WORLD ACCEPTANCE CORP,6.02,1,0.0,0.000000
3,A BARRY RAND,CAMPBELL S CO,1.02,1,0.0,0.000000
4,A BART HOLADAY,MDU RESOURCES GROUP INC,7.04,1,0.0,0.000000
...,...,...,...,...,...,...
13187,ZHONGLI LIU,SMART POWERR CORP,4.01,1,3.0,3.847819
13188,ZI YAO LIM,KULICKE SOFFA INDUSTRIES INC,0.00,1,0.0,0.000000
13189,ZIV SHOSHANI,VISHAY INTERTECHNOLOGY INC,15.99,1,0.0,0.000000
13190,ZUHEIR SOFIA,LANCASTER COLONY CORP,17.99,1,0.0,0.000000


In [19]:
company_directors_details['weight'] = 1 / (
    company_directors_details['log_compensation'].clip(lower=1) *
    company_directors_details['role_score'].clip(lower=1) *
    company_directors_details['service_years'].clip(lower=1)
)

In [23]:
company_directors_details.sample(10)

,director_name,company_name,service_years,is_top_company,role_score,log_compensation,weight
11986,TERESA M RESSEL,INVESCO VALUE MUNICIPAL INCOME TRUST,6.94,1,1.0,6.503809,0.022155
712,ASHTON B CARTER,DELTA AIR LINES INC,4.01,1,0.0,0.000000,0.249377
13025,WILLIAM L JASPER,UNIFI INC,6.96,1,0.0,0.000000,0.143678
750,B THOMAS GOLISANO,EASTMAN KODAK CO,1.99,1,1.0,5.435368,0.092452
4997,JAIME A RAMIREZ,KIMBERLY CLARK CORP,3.01,1,10.0,5.462399,0.006082
8571,MICHAEL F HILTON,NORDSON CORP,4.01,1,10.0,5.987237,0.004165
7547,LINDA W CLEMENT HOLMES,FIFTH THIRD BANCORP,4.01,1000,3.0,5.722696,0.000015
7218,LANCE D MYERS,MAYS J W INC,7.98,1,0.0,0.000000,0.125313
471,ANGELA F BRALY,EXXON MOBIL CORP,5.99,1,3.0,5.577167,0.009978
6597,JOSEPH S SAMBUCO,ALICO INC,1.15,1,0.0,0.000000,0.869565


# 4. Network Analysis

In [24]:
graph = nx.Graph()
people = []
companies = []
for company, director, weight in zip(company_directors_details.company_name, company_directors_details.director_name, company_directors_details.weight):
    graph.add_edge(company, director, weight=weight)
    people.append(director)
    companies.append(company)

In [25]:
biggest_connected_graph = graph.subgraph(list(networkx.connected_components(graph))[0])
print(len(list(networkx.connected_components(graph))))
print(len(biggest_connected_graph.nodes()), len(biggest_connected_graph.edges()))

196
8105 9135


In [27]:
%%time
centrality = pandas.DataFrame({
    'eigen': pandas.Series(nx.eigenvector_centrality(biggest_connected_graph)),
    'degree': pandas.Series(nx.algorithms.degree_centrality(biggest_connected_graph)),
    'closeness_weighted': pandas.Series(nx.closeness_centrality(biggest_connected_graph, distance='weight'))
})
centrality['is_person'] = centrality.index.map(lambda x: x in people)

CPU times: user 10min 27s, sys: 1.96 s, total: 10min 29s
Wall time: 10min 35s


In [37]:
centrality

,eigen,degree,closeness_weighted,is_person
EMERSON ELECTRIC CO,2.452137e-16,0.003949,1.736623,False
A A BUSCH III,1.013264e-17,0.000123,0.931722,True
ACCELERATE DIAGNOSTICS INC,1.454374e-08,0.002962,1.659737,False
A ALEXANDER ARNOLD III,6.009714e-10,0.000123,1.067336,True
WORLD ACCEPTANCE CORP,2.483640e-22,0.001481,0.475338,False
...,...,...,...,...
ZELL B MILLER,5.898645e-21,0.000123,0.328299,True
ZENA SRIVATSA ARNOLD,6.429499e-18,0.000123,1.483584,True
ZENON S NIE,3.225206e-28,0.000123,0.690948,True
ZI YAO LIM,1.056443e-25,0.000123,0.604841,True


In [39]:
centrality[centrality.is_person].reset_index().columns

Index(['index', 'eigen', 'degree', 'closeness_weighted', 'is_person'], dtype='object')

In [83]:
people_df = pandas.merge(centrality[centrality.is_person].reset_index().rename(columns={'index': 'director_name'}), company_director, on="director_name", how="left")
people_df.sample(10)

,director_name,eigen,degree,closeness_weighted,is_person,company_name,service_years,is_top_company
3759,JASON R FUJIMOTO,1.590607e-28,0.000123,0.502561,True,CENTRAL PACIFIC FINANCIAL CORP,2.01,1
1201,CHRISTOPHER L MAPES,7.322897e-15,0.000494,1.808726,True,SMITH A O CORP,1.99,1
3493,JAMES D VAUGHN,1.033325e-01,0.001604,0.360671,True,INVESCO PENNSYLVANIA VALUE MUNICIPAL INCOME TRUST,0.00,1
4803,KENNETH P MANNING,1.789527e-20,0.000123,0.836868,True,SENSIENT TECHNOLOGIES CORP,1.99,1
7119,RICHARD TEMPLETON,1.891039e-15,0.000123,1.749784,True,TEXAS INSTRUMENTS INC,0.00,1
1108,CHIGOZIE O AMADI,5.732578e-31,0.000123,0.756071,True,RCM TECHNOLOGIES INC,1.00,1
8631,TONY ALLEN,6.709125e-18,0.000123,1.729720,True,GRAHAM HOLDINGS CO,4.01,1
8436,THOMAS H HARVEY,1.307810e-17,0.000123,1.707434,True,FIFTH THIRD BANCORP,6.00,1000
8474,THOMAS L BEN,1.618810e-17,0.000123,0.643083,True,SYSCO CORP,0.00,1
3670,JAMES T BUNCH,1.033325e-01,0.001604,0.440492,True,INVESCO SENIOR INCOME TRUST,2.01,1


In [86]:
top_companies_df = people_df[people_df['is_top_company'] == True]
max_closeness = top_companies_df.groupby('company_name')['closeness_weighted'].max().reset_index()
result = pandas.merge(top_companies_df, max_closeness, on=['company_name', 'closeness_weighted'], how='inner')

# Step 4: Return top 10 companies based on closeness
result.sort_values('closeness_weighted', ascending=False).head(10)

,director_name,eigen,degree,closeness_weighted,is_person,company_name,service_years,is_top_company
5,MICHAEL F ROMAN,1.623060e-17,0.000247,1.789213,True,ABBOTT LABORATORIES,2.99,1000
0,ANNE M MULCAHY,3.735174e-15,0.000370,1.782005,True,JOHNSON JOHNSON,3.99,1000
4,MATTHEW J ESPE,6.670313e-18,0.000370,1.755766,True,KORN FERRY,0.00,1000
2,LINDA W CLEMENT HOLMES,1.310710e-17,0.000247,1.707468,True,FIFTH THIRD BANCORP,4.01,1000
6,RICHARD P MCKENNEY,3.580263e-15,0.000247,1.701856,True,UNUM GROUP,8.02,1000
7,SARA MARTINEZ TUCKER,5.556974e-15,0.000247,1.699125,True,SERVICE CORP INTERNATIONAL,6.97,1000
3,MARY J STEELE GUILFOILE,7.591464e-20,0.000370,1.676501,True,VALLEY NATIONAL BANCORP,4.00,1000
1,JOHN Y KIM,3.226523e-18,0.000247,1.588007,True,EVERSOURCE ENERGY,3.01,1000
8,STEVE CAHILLANE,1.169131e-20,0.000247,1.452742,True,KELLANOVA,2.98,1000


In [26]:
# df[df.director_name == 'ELIZABETH KRENTZMAN']

In [27]:
# robust_scaler = sklearn.preprocessing.RobustScaler()
# age_and_demographics_scaled = robust_scaler.fit_transform(people_df[['age', 'log_compensation', 'degree', 'eigen']])

In [28]:
# dbscan = sklearn.cluster.DBSCAN(eps=0.4)
# people_df['cluster_id'] = dbscan.fit_predict(age_and_demographics_scaled)
# people_df.cluster_id.value_counts()

In [29]:
# people_df.plot.scatter(x='age', y='log_compensation', c='cluster_id', cmap="rainbow", s=4)

In [30]:
# people_df.loc['ELIZABETH KRENTZMAN']